In [5]:
import numpy as np
import random as rd
import scipy.stats as sc

l = 20
n = 100000
B = 37.5
xexit = 18.75 + 50

# Water level
data = np.loadtxt('Cumulative_density_function_water_level.txt', delimiter=' ', unpack=True)

def lognorm_params(mode, stddev):
    p = np.poly1d([1, -1, 0, 0, -(stddev/mode)**2])
    r = p.roots
    sol = r[(r.imag == 0) & (r.real > 0)].real
    shape = np.sqrt(np.log(sol))
    scale = mode * sol
    return shape, scale

# Start
Pf = []
for i in range(l):
    # Generate Stochastic Lognormal
    sigma, scale = lognorm_params(25, 0.5)
    D = sc.lognorm.rvs(sigma, 0, scale, size=n)
    
    sigma, scale = lognorm_params(4.5, 0.5)
    d = sc.lognorm.rvs(sigma, 0, scale, size=n)
    
    sigma, scale = lognorm_params(7.52e-4, (0.5 * 7.52e-4))
    k = sc.lognorm.rvs(sigma, 0, scale, size=n)
    
    sigma, scale = lognorm_params(1.00e-6, (0.5 * 1.00e-6))
    kh = sc.lognorm.rvs(sigma, 0, scale, size=n)
    
    sigma, scale = lognorm_params(20, 2)
    Lf = sc.lognorm.rvs(sigma, 0, scale, size=n)
    
    sigma, scale = lognorm_params(0.5, 0.1)
    ich = sc.lognorm.rvs(sigma, 0, scale, size=n)
    
    # Generate Stochastic Normal
    hp = sc.norm.rvs(3.50, 0.1, n)
    
    # Generate water level
    h = []
    for q in range(n):
        w = rd.random()
        h.append(np.interp(w, data[1], data[0]))
    
    # Calc
    lambda_h = ((k * D * d) / kh) ** 0.5
    lambda_ = (lambda_h / (Lf + B + lambda_h)) * np.exp((0.5 * B - xexit) / lambda_h)
    phi_exit = (hp + lambda_ * (h - hp))
    ic = (phi_exit - hp) / d
    Z = ich - ic
    
    fail = 0
    for q in range(n):
        if(Z[q] < 0):
            fail += 1
    
    if(fail > 0):
        Pf.append(fail / n)
    else:
        Pf.append(0)
        
    print(i, "Pf:", Pf[i])
    
print("\nTOTAL Pf:", round(np.average(Pf), 5))
print("SUM Pf:", np.sum(Pf))

0 Pf: 0.00844
1 Pf: 0.00921
2 Pf: 0.00862
3 Pf: 0.00858
4 Pf: 0.00847
5 Pf: 0.00835
6 Pf: 0.00844
7 Pf: 0.00883
8 Pf: 0.00893
9 Pf: 0.00834
10 Pf: 0.00836
11 Pf: 0.00884
12 Pf: 0.00838
13 Pf: 0.0087
14 Pf: 0.00811
15 Pf: 0.0086
16 Pf: 0.00857
17 Pf: 0.00897
18 Pf: 0.00874
19 Pf: 0.00815

TOTAL Pf: 0.00858
SUM Pf: 0.17162999999999998
